In [1]:
import os

os.environ.pop("KAGGLE_USERNAME", None)
os.environ.pop("KAGGLE_KEY", None)

os.environ["KAGGLE_API_TOKEN"] = "KGAT_d9da983d3f79e85e45945ac88b4d001b"

In [2]:
import pandas as pd
from pathlib import Path 

zip_path=Path("home-credit-credit-risk-model-stability.zip")
root=Path("home_credit_data")
train_directory=root/"parquet_files"/"train"
test_directory=root/"parquet_files"/"test"

train=pd.read_parquet(train_directory/"train_base.parquet")
test=pd.read_parquet(test_directory/"test_base.parquet")

In [3]:
static_0_0=pd.read_parquet(train_directory/"train_static_0_0.parquet")
train=train.merge(static_0_0,on="case_id",how="left")

In [4]:
static_cb_0=pd.read_parquet(train_directory/"train_static_cb_0.parquet")
train=train.merge(static_cb_0,on="case_id",how="left")

In [5]:
static_0_1=pd.read_parquet(train_directory/"train_static_0_1.parquet")
train=train.merge(static_0_1,on="case_id",how="left")

In [6]:
tax_registry_a_1=pd.read_parquet(train_directory/"train_tax_registry_a_1.parquet")
tax_registry_a_1.head()

,case_id,amount_4527230A,name_4527232M,num_group1,recorddate_4527225D
0,28631,711.0000,f980a1ea,3,2019-09-13
1,28631,1946.0000,f980a1ea,2,2019-09-13
2,28631,2600.0000,f980a1ea,1,2019-09-13
3,28631,3616.4001,f980a1ea,0,2019-09-13
4,28632,400.0000,5f9b74f5,6,2019-09-13


In [7]:
def aggregate_depth_numeric(df,id_col="case_id",prefix="d1"):
    numeric_cols=df.select_dtypes(include="number").columns.tolist()
    if id_col in numeric_cols:
        numeric_cols.remove(id_col)

    agg=(
    df.groupby(id_col)[numeric_cols].agg(["mean","max","min","std","sum","count"]))

    agg.columns=[f"{prefix}_{c}_{f}" for c,f in agg.columns]
    agg=agg.reset_index()

    return agg

In [8]:
tax_registry_train=aggregate_depth_numeric(tax_registry_a_1,id_col="case_id",prefix="tax_registry")
train=train.merge(tax_registry_train,on="case_id",how="left")

In [9]:
credit_bureau_a_1_0=pd.read_parquet(train_directory/"train_credit_bureau_a_1_0.parquet")
A10credit_bureau_train=aggregate_depth_numeric(credit_bureau_a_1_0,id_col="case_id",prefix="credit_bureau")
train=train.merge(A10credit_bureau_train,on="case_id",how="left")

In [10]:
applprev_2=pd.read_parquet(train_directory/"train_applprev_2.parquet")
applprev_train=aggregate_depth_numeric(applprev_2,id_col="case_id",prefix="applprev")
train=train.merge(applprev_train,on="case_id",how="left")

In [11]:
applprev_1_0=pd.read_parquet(train_directory/"train_applprev_1_0.parquet")
applprev10_train=aggregate_depth_numeric(applprev_1_0,id_col="case_id",prefix="applprev10")
train=train.merge(applprev10_train,on="case_id",how="left")

In [12]:
credit_bureau_a_1_1=pd.read_parquet(train_directory/"train_credit_bureau_a_1_1.parquet")
A11credit_bureau_train=aggregate_depth_numeric(credit_bureau_a_1_1,id_col="case_id",prefix="credit_bureauA11")
train=train.merge(A11credit_bureau_train,on="case_id",how="left")

In [13]:
credit_bureau_a_1_2=pd.read_parquet(train_directory/"train_credit_bureau_a_1_2.parquet")
A12credit_bureau_train=aggregate_depth_numeric(credit_bureau_a_1_2,id_col="case_id",prefix="credit_bureauA12")
train=train.merge(A12credit_bureau_train,on="case_id",how="left")

In [14]:
credit_bureau_a_2_3=pd.read_parquet(train_directory/"train_credit_bureau_a_2_3.parquet")
A23credit_bureau_train=aggregate_depth_numeric(credit_bureau_a_2_3,id_col="case_id",prefix="credit_bureauA23")
train=train.merge(A23credit_bureau_train,on="case_id",how="left")

In [ ]:
import numpy as np

X=train.drop(columns=["target","case_id","date_decision"])
y=train["target"]
numeric=(X.select_dtypes(include=["number"]).replace([np.inf,-np.inf],np.nan).fillna(0.0).astype("float32"))

In [ ]:
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,cross_val_score

predict=np.zeros(len(numeric))

for fold,(tr_idx,test_idx) in enumerate(skf.split(numeric,y)):
    X_train,X_test=numeric.iloc[tr_idx],numeric.iloc[test_idx]
    y_train,y_test=y[tr_idx],y[test_idx]

    model=CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        learning_rate=0.05,
        depth=6,
        iterations=1000,
        random_seed=42,
        verbose=False)

    model.fit(X_train,y_train,eval_set=(X_test,y_test),verbose=False)
    predict[test_idx]=model.predict_proba(X_test)[:,1]

cv_auc=roc_auc_score(y,predict)
print("CV AUC:",cv_auc)

In [16]:
from sklearn.model_selection import KFold

kf=KFold(n_splits=5,shuffle=True,random_state=42)

In [17]:
skf=KFold(n_splits=3,shuffle=True,random_state=42)

In [18]:
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,cross_val_score

In [19]:
from sklearn.model_selection import train_test_split

week_num=train["WEEK_NUM"].values

X_train1,X_test1,y_train1,y_test1,week_train,week_test=train_test_split(
    numeric,y,week_num,test_size=0.2,random_state=42,stratify=y)

model1=CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    learning_rate=0.05,
    depth=6,
    iterations=500,
    random_seed=42,
    verbose=False)

model1.fit(X_train1,y_train1,eval_set=(X_test1,y_test1),verbose=False)
predict1=model1.predict_proba(X_test1)[:,1]
auc1=roc_auc_score(y_test1,predict1)
print("AUC score:",auc1)

AUC score: 0.8210179155096572


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score

def metric(y,y_pred,week_number):
    y=np.asarray(y).ravel()
    y_pred=np.asarray(y_pred).ravel()
    week_number=np.asarray(week_number).ravel()

    if not (len(y)==len(y_pred)==len(week_number)):
        raise ValueError("y, y_pred, and week_number must have the same length")
    
    df=pd.DataFrame({
        "y":y,
        "y_pred":y_pred,
        "week":week_number})

    weekly=[]
    for w,group in df.groupby("week"):
        if group["y"].nunique()<2:
            continue
        auc=roc_auc_score(group["y"],group["y_pred"])
        gini=2*auc-1
        weekly.append((w,gini))

    if not weekly:
        return np.nan

    weeks,ginis=map(np.array,zip(*weekly))

    order=np.argsort(weeks)
    weeks=weeks[order]
    ginis=ginis[order]

    #gini=a*week+b
    Xw=weeks.reshape(-1,1)
    reg=LinearRegression()
    reg.fit(Xw,ginis)

    a=float(reg.coef_[0])
    b=float(reg.intercept_)

    gini_hat=reg.predict(Xw)
    residuals=ginis-gini_hat

    mean_gini=ginis.mean()
    falling_rate=min(0.0,a)
    residuals_std=residuals.std(ddof=0)

    score=mean_gini+88.0*falling_rate-0.5*residuals_std

    return score

In [21]:
stability_score=metric(y_test1,predict1,week_test)
print("Stability metric:",stability_score)

Stability metric: 0.5982852028307518


In [22]:
important_features=pd.DataFrame({
    "feature":numeric.columns,
    "importance":model1.get_feature_importance()}).sort_values("importance",ascending=False)

important_features.head(20)

,feature,importance
155,pmtaverage_3A,3.765699
118,pmtnum_254L_x,2.134299
80,mobilephncnt_593L_x,1.985911
15,avgdpdtolclosure24_3658938P_x,1.768169
131,days120_123L,1.718464
112,numrejects9m_859L_x,1.666787
161,pmtscount_423L,1.614220
162,pmtssum_45A,1.520219
1057,credit_bureauA11_totalamount_6A_max,1.442518
286,price_1097A_y,1.351794


In [23]:
log_cols=[
    "pmtaverage_3A",
    "pmtsum_45A",
    "credit_bureauA11_totalamount_6A_max",
    "credit_bureauA11_totalamount_6A_mean",
    "credit_bureauA11_totalamount_6A_sum",
    "tax_registry_amount_4527230A_sum",
    "price_1097A_x",
    "price_1097A_y"]

for col in log_cols:
    if col in numeric.columns:
        numeric[f"{col}_log1p"]=np.log1p(numeric[col].clip(lower=0))